# Rapid Adiabatic Passage Simulations for $\text{CH}_4$

In this notebook, we will try to reproduce calculations and simulation presented by Rainer Beck *et al.* in *J.Chem.Phys* **140**, 034321 (2014) for preparation of quantum state specific reactants in a molecular beams by rapid adiabatic passage. 

In [1]:
# importing some paraphernelia 
import numpy as np
import matplotlib as mpl
mpl.rc('text', usetex=True)
mpl.rc('font', family='sans-serif', serif='computer modern')
import matplotlib.pyplot as plt
from scipy.constants import c, mu_0, epsilon_0, h, hbar

$$
w_{x}(0) = \frac{f\lambda}{r\pi}
$$

$$
w_{x}(z) = w_{x}(0)\sqrt{1+\frac{z^2}{z_R^2}}
$$

$$
z_{R} = \frac{\pi w_{x}(0)^2}{\lambda}
$$

$$
R(z) = z + \frac{z_{R}^2}{z}
$$

In [2]:
!source /opt/intel/oneapi/setvars.sh

 
:: initializing oneAPI environment ...
   bash: BASH_VERSION = 5.1.16(1)-release
   args: Using "$@" for setvars.sh arguments: 
:: advisor -- latest
:: ccl -- latest
:: compiler -- latest
:: dal -- latest
:: debugger -- latest
:: dev-utilities -- latest
:: dnnl -- latest
:: dpcpp-ct -- latest
:: dpl -- latest
:: ipp -- latest
:: ippcp -- latest
:: mkl -- latest
:: mpi -- latest
:: tbb -- latest
:: vtune -- latest
:: oneAPI environment initialized ::
 


In [3]:
import os
import sys
sys.path.append('..')
from rap_mb import *

ImportError: libsvml.so: cannot open shared object file: No such file or directory

In [ ]:
wl_nm = 3300 # nm
r = 0.2      # cm   
focal_lengths = [10, 25.4, 40, 65, 96] #cm
z = np.linspace(-20, 20, 1000) # cm

In [ ]:
fig, ax = plt.subplots(figsize=(5,4))
for f in focal_lengths:
    params = f, wl_nm, r
    label = '{f:2.1f} cm'.format(f=f)
    w0 = wl_nm*1e-7*f/(np.pi*r)
    ax.plot(z, abs(R(z, w0, wl_nm)), label=label)
ax.set_ylim(0, 60)
ax.set_xlim(-20,20)
ax.set_xlabel('$z$ [cm]')
ax.legend(bbox_to_anchor=[.75,0.75], frameon=False, title=r'$f$')
ax.set_ylabel(r'$|R(z)|$ [cm]')

In [ ]:
fig, ax = plt.subplots(figsize=(5,4))
for f in focal_lengths:
    params = f, wl_nm, r
    label = '{f:2.1f} cm'.format(f=f)
    ax.plot(z, w(z, wl_nm, r, f, focused=True)*10, label=label)
ax.set_ylim(0, 2.0)
ax.set_xlim(-20,20)
ax.set_xlabel(r'$z$ [cm]')
ax.legend(bbox_to_anchor=[.5,0.5], frameon=False, title=r'$f$')
ax.set_ylabel(r'$w_{x}(z)$ [mm]')

### Sweep rates

$$
\frac{d\Phi}{dt} = \frac{v_x^2\omega_{21}}{R(z)c}
$$

$$
T_{trans} = \frac{2w_{x}(z)}{v_{x}}
$$

The total sweep experienced by the molecule from the curved wavefronts can be calculated as 
$$
\frac{d\Phi}{dt}T_{trans}
$$

The frequency sweep experienced by the molecule at a time t is given by  
$$
\Delta(t) = \omega_{21} - \omega_{L} - \frac{d\Phi}{dt}t
$$

where $\omega_{21}$ is the frequency of the transition, $\omega_L$ the laser frequency. So, if the detuning is zero *i.e.* if the laser is resonant with the transition and, if there is no divergence in the molecular beam, the sweep is only due to the curved wavefronts.


However, there's usually an angular divergence associated with the molecular beam and, this gives rise to a Doppler shift associated with the component of velocity that is parallel ($v_{z}$) to the direction of propagation of the laser field.

$$
\Delta(t) = \frac{v_z \omega_{21}}{c} - \frac{v_x^2\omega_{21}}{R(z)c}t
$$

In [ ]:
vx = 2056
z = np.linspace(0,20, 2000)

In [ ]:
fig, ax = plt.subplots(figsize=(5,4))
for f in focal_lengths:
    params = wl_nm, r, f
    label = '{f:2.1f} cm'.format(f=f)
    ax.plot(z, sweep_rate(z, vx, params), label=label)
ax.set_xlim(0, 20)
ax.set_ylim(0, 5e14)
ax.set_xlabel('$z$ [cm]')
ax.legend(bbox_to_anchor=[.5,0.65], frameon=False,title=r'$f$')
ax.set_ylabel(r'${d\phi}/{dt}$ [Hz s$^{-1}$]')

In [ ]:
fig, ax = plt.subplots(figsize=(5,4))
for f in focal_lengths:
    params = wl_nm, r, f
    label = '{f:2.1f} cm'.format(f=f)
    tr = transit_time(z,vx,params)
    ax.plot(z, tr*sweep_rate(z, vx, params), label=label)
ax.set_xlim(0,20)
ax.set_ylim(0,1.6e8)
ax.set_xlabel('$z$ [cm]')
ax.legend(bbox_to_anchor=[.5,0.65], frameon=False,title=r'$f$')
ax.set_ylabel(r'$T_\textit{\tiny trans} \ {d\phi}/{dt}$ [Hz s$^{-1}$]')

In [ ]:
dipole = 1.8e-31 # C-m
P = 0.5
Nz = 100
v = 2000
delta_v = 5
divergence = 0.250
z = np.linspace(0, 20, Nz)
fig, ax = plt.subplots(figsize=(5,4))
for f in focal_lengths:
    params = wl_nm, r,f
    label = '{f:2.1f}'.format(f=f)
    vel_xyz = sample_velocities(v, delta_v, divergence)
    E0 = Exy(0, z, 0.5, vel_xyz, params)
    omega_r = E0*dipole/hbar
    ax.plot(z, omega_r/1e6, label=label)
ax.set_xlim(0,20)
ax.set_ylim(0,85.0)
ax.set_xlabel('$z$ [cm]')
ax.legend(bbox_to_anchor=[.85,0.85], frameon=False,title=r'$f$ [cm]')
ax.set_ylabel(r'$\Omega(z)$ MHz')

In [ ]:
Nt = 200
Nz = 100
N_sample = 10000
t = np.linspace(-1e-6, 1e-6, Nt)
z = np.linspace(0, 100, Nz)
Z, T = np.meshgrid(z, t)
E = np.empty((Nt, Nz), dtype=np.float64)
velocities = np.empty((N_sample, 3), dtype=np.float64)
for i in range(N_sample):
    vel_xyz = sample_velocities(v, delta_v, divergence)
    velocities[i,:] = vel_xyz
    E += Exy(T, Z, 0.5, vel_xyz, params)/1e3
E = E/N_sample

fig, ax = plt.subplots(figsize=(5,4))
cs = ax.contourf(T/1e-6, Z, E, levels=20)
cbar = plt.colorbar(cs, spacing='proportional')
cbar.ax.set_title(r'$E$ [kV/m]')
fig.tight_layout()
ax.set_xlabel(r'$t$ [$\mu$s]')
ax.set_ylabel(r'$z$ [cm]')
fig.suptitle(r'$E_{avg}$ felt by molecules')

In [ ]:
fig, ax = plt.subplots(3,1, figsize=(7, 6))
ax[0].set_title(r'$V_{x}$')
ax[1].set_title(r'$V_{y}$')
ax[2].set_title(r'$V_{z}$')
ax[0].hist(velocities[:,0], bins=50)
ax[1].hist(velocities[:,1],bins=50)
ax[2].hist(velocities[:,2],bins=50)
fig.tight_layout()

## Optical Bloch Equations

Now, let's solve the optical Bloch equations for the two-level system.

$$
\begin{align*}

\frac{dU}{dt} &= -V \\

\frac{dV}{dt} &= \Delta U + \Omega W \\

\frac{dW}{dt} &= -\Omega V

\end{align*}
$$


In [ ]:
from scipy.constants import k, N_A

T_para = 20 # K 
T_perp = 5   # K
M = 28.01 *1e-3/N_A # kg
v0 = 2000
v = np.linspace(1800, 2200, 100)
f = lambda T : np.exp(-M*(v-v0)**2/(k*T))
plt.plot(v, f(T_para))